In [2]:
from basic.api_key import ensure_api_key
ensure_api_key()
from openai import OpenAI
client = OpenAI()

OpenAI API key has been configured.


In [3]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="Write a Python code that is printing the first 10 Fibonacci numbers. Reply to me in pure Python. DO NOT USE MARKDOWN FORMATTING.",
)
print(response.output_text)

[2025-09-14 16:52:04 - httpx:1025 - INFO] HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


def fibonacci(n):
    fib_sequence = []
    a, b = 0, 1
    for _ in range(n):
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

first_10_fibonacci = fibonacci(10)
for number in first_10_fibonacci:
    print(number)


# Is it going to work?

In [5]:
exec(response.output_text)

0
1
1
2
3
5
8
13
21
34


# Other formats

## CSV

In [8]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="Write for me a content of a CSV file with 20 rows and 3 columns: name, age, city. Use only Polish names and cities. Include headers. DO NOT USE MARKDOWN FORMATTING. Just the CSV content.",
)
print(response.output_text)

[2025-09-14 16:56:15 - httpx:1025 - INFO] HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


name,age,city  
Jan,32,Warszawa  
Kasia,28,Kraków  
Marek,45,Łódź  
Zofia,37,Gdańsk  
Andrzej,29,Poznań  
Agnieszka,41,Wrocław  
Piotr,22,Szczecin  
Monika,34,Lublin  
Tomasz,50,Rzeszów  
Karolina,27,Zielona Góra  
Jakub,30,Bydgoszcz  
Natalia,33,Olsztyn  
Adam,26,Katowice  
Ewa,39,Opole  
Krzysztof,53,Gdynia  
Justyna,31,Toruń  
Robert,48,Radom  
Aleksandra,24,Sosnowiec  
Michał,36,Chorzów  
Magdalena,40,Elbląg  


In [12]:
from io import StringIO
import pandas as pd
pd.read_csv(StringIO(response.output_text))

,name,age,city
0,Jan,32,Warszawa
1,Kasia,28,Kraków
2,Marek,45,Łódź
3,Zofia,37,Gdańsk
4,Andrzej,29,Poznań
5,Agnieszka,41,Wrocław
6,Piotr,22,Szczecin
7,Monika,34,Lublin
8,Tomasz,50,Rzeszów
9,Karolina,27,Zielona Góra


## JSON

In [13]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="Generate a fictitious person. Reply to me in JSON format. DO NOT USE MARKDOWN FORMATTING. Just the JSON content. Please include the following fields: first_name (string), last_name (string), age (number), city (string), email (string).",
)
print(response.output_text)

[2025-09-14 17:02:04 - httpx:1025 - INFO] HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


{
  "first_name": "Lila",
  "last_name": "Winston",
  "age": 29,
  "city": "Austin",
  "email": "lila.winston@example.com"
}


### Use Pydantic

In [21]:
from pydantic import BaseModel, Field
import json

class Person(BaseModel):
    first_name: str = Field(description='First name of the person. It should be a common Polish first name.')
    last_name: str = Field(description='Last name of the person. It should be a common Polish last name.')
    age: int = Field(description='Age of the person. It should be a number between 0 and 120.')
    city: str = Field(description='City where the person lives. It should be a common Polish city.')
    email: str = Field(description='Email address of the person. It should be a valid email format in domain wp.pl, onet.pl or o2.pl.')

schema = json.dumps(Person.model_json_schema())
print(schema)

{"properties": {"first_name": {"description": "First name of the person. It should be a common Polish first name.", "title": "First Name", "type": "string"}, "last_name": {"description": "Last name of the person. It should be a common Polish last name.", "title": "Last Name", "type": "string"}, "age": {"description": "Age of the person. It should be a number between 0 and 120.", "title": "Age", "type": "integer"}, "city": {"description": "City where the person lives. It should be a common Polish city.", "title": "City", "type": "string"}, "email": {"description": "Email address of the person. It should be a valid email format in domain wp.pl, onet.pl or o2.pl.", "title": "Email", "type": "string"}}, "required": ["first_name", "last_name", "age", "city", "email"], "title": "Person", "type": "object"}


In [23]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="Generate a fictitious person. Reply to me in JSON format. DO NOT USE MARKDOWN FORMATTING. Just the JSON content. follow this schema: " + schema,
)
print(response.output_text)

[2025-09-14 17:15:14 - httpx:1025 - INFO] HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


{
  "first_name": "Katarzyna",
  "last_name": "Nowak",
  "age": 28,
  "city": "Kraków",
  "email": "katarzyna.nowak@wp.pl"
}


In [24]:
person = Person.model_validate_json(response.output_text)
person

Person(first_name='Katarzyna', last_name='Nowak', age=28, city='Kraków', email='katarzyna.nowak@wp.pl')

### With OpenAI SDK

In [34]:
# Here's a small change
response = client.responses.parse(
    model="gpt-4o-mini",
    input="Generate a fictitious person.",
    text_format=Person # And we add this
)

[2025-09-14 17:21:31 - httpx:1025 - INFO] HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


In [35]:
response.output_text

'{"first_name":"Marek","last_name":"Kowalski","age":28,"city":"Kraków","email":"marek.kowalski@wp.pl"}'

In [36]:
person = Person.model_validate_json(response.output_text)
person

Person(first_name='Marek', last_name='Kowalski', age=28, city='Kraków', email='marek.kowalski@wp.pl')